## Weaviate Query Agent with Agno

This notebook will show you how to define the Weaviate Query Agent as a tool through Agno.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. Install Agno with `pip install agno`. We're using version `1.1.8` for this notebook
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.

### Agno Resources
Learn more about Agno and how to build multimodal agents [here](https://docs.agno.com/introduction)!


### Import libraries and keys

In [1]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from textwrap import dedent

In [2]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

### Define Query Agent function

In [3]:
def query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    # connect to your Weaviate Cloud instance
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"), 
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY") # add the API key to the model provider from your Weaviate collection
        }
    )

    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["WeaviateBlogChunks"] 
    )
    return query_agent.run(query).final_answer

### Define agent

In [9]:
agent = Agent(
    model=OpenAIChat(id="gpt-4"),
    instructions=dedent("""\
        You are a helpful assistant that can search through Weaviate's documentation and blog posts to answer questions.
        
        When answering questions:
        - Search the Weaviate knowledge base using the provided query tool
        - Provide clear, accurate technical information
        - Include relevant code examples when appropriate
        - Break down complex concepts into understandable explanations
        - If you're unsure about something, say so rather than making assumptions
        
        Focus on helping users understand and work with Weaviate effectively.\
    """),
    tools=[query_agent_request],
    show_tool_calls=True,
    markdown=True,
)

### Query time

In [11]:
response = agent.print_response("How do I run Weaviate with Docker?", stream=True)

Output()

/usr/local/lib/python3.11/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to 
Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(

/usr/local/Cellar/python@3.11/3.11.6/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/selector_even
ts.py:864: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=112 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
ResourceWarning: Enable tracemalloc to get the object allocation traceback


None


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x11921f150>
